# Preparatory steps



In [ ]:
# access data from the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# alternative: upload files
# from google.colab import files
# uploaded = files.upload()

### Installing necessary packages

Install the necessary bert packages and Tensorflow 1.15.2

In [ ]:
!pip uninstall gast
!pip install gast==0.2.2
!pip install sentencepiece
!pip install bert-tensorflow==1.0.1
!pip install pandas==0.24

After restarting the runtime as suggested, run following:

In [ ]:
%tensorflow_version 1.x

In [ ]:
# for importing modules in .py files, insert the working directory
import sys
sys.path.insert(0,'/content/drive/My Drive/AIandLaw/CodeAndModel/')

# Predict

This can be used to predict the labels (hateful or not) of new data with existing models.
Before you run it, you have to change the following input variables;
- data_path: Location of the new data.
- bert_model_path: Location of the BERT model.
- xgb_model_path: Location of the predictions model.
- bert_output_path = Location where the BERT features should be saved.
- predictions_output_path = Location where the predictions are saved.
The data_path should point to a csv file with a column called "Comment_text".
BERT is a huge machine learning model and it takes quite long to generate features with it, that why the BERT
features are stored. If for some reason an error happens afterwards, the BERT features don't have to be computed again.
The output is a csv file with three columns: the original comment, the predicted probability that is is a hateful
comment and the binary prediction (True if the predicted probability is greater than 0.5).
To execute the script you just need to change the variables in the "User Input" section and then run the script.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf

from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

from bert_functions_modified import *
from functions_modified import *

In [ ]:
# --------- User Input ----------------------------
data_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/dataset/test_data_hate_speech.csv"
bert_model_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/bert_model_updated.h5"
xgb_model_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/xgboost_model.dat"
bert_output_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/bert_output.csv"
predictions_output_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/predictions.csv"

In [ ]:
new_data = pd.read_csv(data_path)

In [ ]:
sess = tf.Session()
max_seq_length = 256
bert_model = build_model(max_seq_length, bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
initialize_vars(sess)
bert_model.load_weights(bert_model_path)

### Instantly evaluating single comment without XGBoost 

For testing purposes. Enter a text in the INPUT TEXT cell and run it to instantly detect hate speech without needing to save it as .csv first.

Note: need to build the model first from previous cells

In [ ]:
tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", sess)

In [ ]:
def evaluateTextForHateSpeech(text, tokenizer):
  text = str(text)
  text = text = [' '.join(str(text).split()[0:max_seq_length])]
  text = np.array(text, dtype=object)[:, np.newaxis]
  pseudo_label = [1] * len(text)

  example = convert_text_to_examples(text, pseudo_label)
  (input_ids, input_masks, segment_ids, labels) = convert_examples_to_features(tokenizer, example, max_seq_length=max_seq_length)

  prediction = bert_model.predict([input_ids, input_masks, segment_ids])

  if prediction > 0.5:
    return "Hate speech detected"
  else:
    return "No hate speech detected"

In [ ]:
#INPUT TEXT
textToBeEvaluated = "White people are trashy."

In [ ]:
evaluateTextForHateSpeech(textToBeEvaluated, tokenizer)

### Create dataset for BERT

In [ ]:
# create datasets for BERT
text = new_data['Comment_text'].tolist()
text = [' '.join(str(t).split()[0:max_seq_length]) for t in text]
text = np.array(text, dtype=object)[:, np.newaxis]
pseudo_label = [1] * len(text)

tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", sess)

In [ ]:
# Convert data to InputExample format
examples = convert_text_to_examples(text, pseudo_label)

In [ ]:
# Convert to features
(input_ids, input_masks, segment_ids, labels
) = convert_examples_to_features(tokenizer, examples, max_seq_length=max_seq_length)

In [ ]:
bert_data, bert_features = get_bert_features([input_ids, input_masks, segment_ids], bert_model)

In [ ]:
bert_data = new_data.reset_index().join(bert_data).set_index('index')
bert_data.to_csv(bert_output_path)

### Performace on test set

In [ ]:
#restart
bert_data=pd.read_csv(bert_output_path)

In [ ]:
xgb_model = joblib.load(xgb_model_path)
bert_data = pd.read_csv(bert_output_path)
bert_data['probabilities'] = xgb_model.predict(bert_data.loc[:, bert_features])

In [ ]:
def plot_results(X_test, threshold):
  X_test['predictions'] = X_test['probabilities'] > threshold
  cf_matrix = confusion_matrix(X_test['Hateful_or_not'], X_test['predictions'])
  labels = ['True Negative','False Positive','False Negative','True Positive']
  categories = ['Non-hateful', 'Hateful']
  make_confusion_matrix(cf_matrix, 
                        group_names=labels,
                        categories=categories, 
                        cmap='Blues')
  # add plot
  evaluate_model(X_test['Hateful_or_not'], X_test['probabilities'], plot=True)
  print(matthews_corrcoef(X_test['Hateful_or_not'], X_test['predictions']))

In [ ]:
plot_results(bert_data, 0.9)

### Output predictions

In [ ]:
predictions = new_data
predictions['prediciton_score'] = bert_data['probabilities']
predictions['predictions'] = bert_data['probabilities'] > 0.9

In [ ]:
predictions.sample(8)

In [ ]:
predictions.to_csv(predictions_output_path)